# DSPy RAG Example with Azure OpenAI

## Setup

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import dspy
import ujson
import random
import dspy.evaluate
import dspy.retrievers

In [ ]:
# Load environment variables from .env file
load_dotenv()

True

## Setup Tracing with MLFlow

In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy-Azure-Eval-Example")

# Enable tracing.
mlflow.dspy.autolog()

## Setup LM Configuration

In [65]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

lm = dspy.LM(
    model=f"azure/gpt-4.1",
    base_url=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_ad_token_provider=token_provider,
)
dspy.configure(lm=lm)

In [66]:
# Test the LM configuration.
lm("Say hello to the world!")

['Hello, world! 🌍']

Trace(trace_id=3bb96e9954d44bc0b088f3bf9b44fffd)

## Start with DSPy modules

In [67]:
qa = dspy.Predict("question: str -> response: str")
response = qa(question="What are high memory and low memory on Linux?")

print(response)

Prediction(
    response='On Linux, "high memory" and "low memory" refer to different regions of physical RAM as seen by the kernel, especially on 32-bit systems:\n\n- **Low Memory**: This is the portion of physical RAM that the Linux kernel can directly address and map into its own address space. On 32-bit x86 systems, this is typically the first ~896 MB of RAM (from 0x00000000 to 0x37FFFFFF). The kernel and most kernel data structures reside here, and it is directly accessible by the kernel at all times.\n\n- **High Memory**: This is any physical RAM above the low memory threshold (above ~896 MB on 32-bit x86). The kernel cannot directly map all of high memory into its address space due to address space limitations. Instead, it must temporarily map high memory pages when it needs to access them. High memory is used primarily for user-space processes and file caches.\n\nOn 64-bit systems, the distinction is largely irrelevant because the kernel can address much more memory directly, s

Trace(trace_id=53d56f974e764fafac77cdb12bc35d03)

In [68]:
cot = dspy.ChainOfThought("question: str -> response: str")
cot(question="Should curly braces appear on the same line as the function name in C#?")

Prediction(
    reasoning='In C#, the placement of curly braces is a matter of style, but the most widely accepted convention (as recommended by Microsoft and used in most C# codebases) is to place the opening curly brace on a new line after the function declaration. This is known as the "Allman style." However, some developers may use the "K&R style," where the opening brace appears on the same line as the function name, but this is less common in C#.',
    response='No, in C#, the standard convention is to place the opening curly brace on a new line after the function name, not on the same line.'
)

Trace(trace_id=dcfe007473e440c1bb558dd2175e811b)

## Evaluate with Examples

In [ ]:
! wget https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl -O ragqa_arena_tech_examples.jsonl

--2025-07-13 13:18:05--  https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.90, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /api/resolve-cache/models/dspy/cache/ef6a5e72a98b47cef31574a400fea8fe149559a3/ragqa_arena_tech_examples.jsonl?%2Fdspy%2Fcache%2Fresolve%2Fmain%2Fragqa_arena_tech_examples.jsonl=&etag=%2236aea6cafdd206141e4ce7a897b5c38f0be9d785%22 [following]
--2025-07-13 13:18:05--  https://huggingface.co/api/resolve-cache/models/dspy/cache/ef6a5e72a98b47cef31574a400fea8fe149559a3/ragqa_arena_tech_examples.jsonl?%2Fdspy%2Fcache%2Fresolve%2Fmain%2Fragqa_arena_tech_examples.jsonl=&etag=%2236aea6cafdd206141e4ce7a897b5c38f0be9d785%22
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 200 OK
Length: 1522638 (1.5M) [text/plain]

In [69]:
with open("ragqa_arena_tech_examples.jsonl", "r") as f:
    data = [ujson.loads(line) for line in f]

data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [70]:
data = [dspy.Example(**d).with_inputs("question") for d in data]
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [71]:
random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]
len(trainset), len(devset), len(testset)

(200, 300, 500)

In [72]:
metric = dspy.evaluate.SemanticF1(decompositional=True)

pred = cot(**example.inputs())

score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 why are my text messages coming up as maybe?

Gold Response: 	 This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". 

However, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.

Predicted Response: 	 Your text messages are showing up as "Maybe: [Name]" because your phone is trying to guess the identity of the sender based on information it finds in your emails, messages, or other apps. This happens when the sender's number isn't saved in your contacts, or if the contact information is incomplete. To stop seeing "Maybe," save the sender's number to your contacts with the correct name.

Semantic F1 Score: 0.40


[Trace(trace_id=3d2b2ab0be6b4e53aa0a5be4dba29134), Trace(trace_id=324d8615c2734db985bc4e7b9a5189da)]

In [22]:
# Define the evaluator.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24, display_progress=True, display_table=2)

In [ ]:
# Evaluate the Chain-of-Thought program.
evaluate(cot)

Average Metric: 128.09 / 300 (42.7%): 100%|██████████| 300/300 [01:49<00:00,  2.74it/s]

2025/07/13 13:33:20 INFO dspy.evaluate.evaluate: Average Metric: 128.08507795163482 / 300 (42.7%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],The choice between C and C++ depends on the requirements of the pr...,Use C over C++ when: - You need maximum portability and minimal ru...,✔️ [0.154]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",Storing images in a git repository is a nuanced topic. Git is opti...,"In general, it's best to avoid storing large or numerous images di...",✔️ [0.444]


42.7

[Trace(trace_id=3aa8008a0e6a4fbfb0d952ce1d56f5a9), Trace(trace_id=1e829c892ccf40e4bc647ee59d4b60ca), Trace(trace_id=10ef9eaaa05c468f8ee5d8e891078756), Trace(trace_id=f1a6836673f74e67a67a1b0db893674b), Trace(trace_id=1e4a525850f543efab4056eb4987cc06), Trace(trace_id=b2047b38fb364889bc378696f8e3699f), Trace(trace_id=5ad7480f753345319fa0cc2633cfbf00), Trace(trace_id=4a5d5bff8d4344daa0bcb258447896fa), Trace(trace_id=3e320cc514d34636a540cf9a7031c1ad), Trace(trace_id=e75dd66c41d549b0aa5eab404ad3ff88)]

## Custom DSPy Module

In [26]:
! wget https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl -O ragqa_arena_tech_corpus.jsonl

--2025-07-13 13:38:47--  https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl
Resolving huggingface.co (huggingface.co)... 108.138.246.67, 108.138.246.79, 108.138.246.85, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.67|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3c/d1/3cd17edd9d00b6cb87d90d566d09ca0a0d2242d61c9c36b9e3c6c82c41a34479/a5d0eff9ad3fece29c97d59ccf24ae80cd6317f8781c218ff355be922056fe7d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ragqa_arena_tech_corpus.jsonl%3B+filename%3D%22ragqa_arena_tech_corpus.jsonl%22%3B&Expires=1752442728&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjQ0MjcyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNjL2QxLzNjZDE3ZWRkOWQwMGI2Y2I4N2Q5MGQ1NjZkMDljYTBhMGQyMjQyZDYxYzljMzZiOWUzYzZjODJjNDFhMzQ0NzkvYTVkMGVmZjlhZDNmZWNlMjljOTdkNTljY2YyNGFlODBjZDYzMTdmODc4MWMyMThmZjM1N

In [ ]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
azure_ad_token = token_provider()

embedder = dspy.Embedder(
    f'azure/text-embedding-3-small',
    api_base=os.environ["AZURE_OPENAI_EMBEDDING_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
    azure_ad_token=azure_ad_token,
    dimensions=512,
)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Loaded 28436 documents. Will encode them below.
Training a 32-byte FAISS index with 337 partitions, based on 28436 x 512-dim embeddings


In [41]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [ ]:
rag = RAG()
rag(question="What are high memory and low memory on Linux?")

Prediction(
    reasoning="The context provides several explanations about high memory (highmem) and low memory (lowmem) in Linux, especially in the context of 32-bit systems. Low memory refers to the portion of physical memory that is permanently mapped into the kernel's address space, making it directly accessible to the kernel. High memory, on the other hand, is the portion of physical memory that is not permanently mapped and requires special handling (such as temporary mapping) for the kernel to access it. This distinction arises because, on 32-bit systems, the kernel's address space is limited (typically 1GB out of 4GB), so not all physical memory can be mapped at once. High memory is generally used for user-space applications and the page cache, while low memory is critical for kernel operations and data structures. The kernel must use functions like kmap and kunmap to access high memory pages.",
    response='On Linux, especially on 32-bit systems, "low memory" refers to the po

Trace(trace_id=fcf4d8d5fac141239ffb15192d7c36db)

In [43]:
evaluate(RAG())

Average Metric: 168.15 / 300 (56.1%): 100%|██████████| 300/300 [02:07<00:00,  2.36it/s]

2025/07/13 14:15:11 INFO dspy.evaluate.evaluate: Average Metric: 168.1502347173502 / 300 (56.1%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],The context provides several perspectives on when to use C versus ...,Use C over C++ when: - You are working in embedded systems or on p...,✔️ [0.429]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",The context provides a range of opinions and technical considerati...,"Images can be stored in a Git repository, especially if they are e...",✔️ [0.500]


56.05

[Trace(trace_id=9873726e5d4a4298962f016079ed5b0e), Trace(trace_id=20d497ebbd584d37a49bfd1a3dea91f4), Trace(trace_id=2fd7c8c4043947e3a8d10cc2ee7866be), Trace(trace_id=295d41ac5673491c80178e36dc9db3c2), Trace(trace_id=2dac4f272b05405fbf07918081300543), Trace(trace_id=9923980c11a84b4b9f9689d3c433578f), Trace(trace_id=2aeee7366b2342cb81a8a233170f8353), Trace(trace_id=50f637c9bbca46e8958a10d23e3902c0), Trace(trace_id=a7b042890fd14eb78e3c3f18d5d55ca0), Trace(trace_id=6a2995f1ddec4b1ab7a5c06fdddc11de)]

## Optimize using `MIPROv2`

In [44]:
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(RAG(), trainset=trainset,
                           max_bootstrapped_demos=2, max_labeled_demos=2,
                           requires_permission_to_run=False)

2025/07/13 14:17:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 160

2025/07/13 14:17:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/13 14:17:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/13 14:17:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


 10%|█         | 4/40 [00:41<06:10, 10.30s/it]


Bootstrapped 2 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/12


 20%|██        | 8/40 [01:07<04:29,  8.43s/it]


...

2025/07/13 14:32:56 INFO dspy.evaluate.evaluate: Average Metric: 19.59549286044601 / 35 (56.0%)
2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 55.99 on minibatch of size 35 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [60.5, 49.37, 56.45, 56.69, 54.13, 57.35, 59.21, 54.66, 51.92, 54.56, 50.87, 57.91, 55.51, 55.75, 58.66, 57.4, 55.99]
2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [55.15, 56.26, 57.41, 56.38]
2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 57.41
2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/13 14:32:56 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 22 / 23 - Minibatch ==



Average Metric: 22.80 / 35 (65.1%): 100%|██████████| 35/35 [00:20<00:00,  1.69it/s]

2025/07/13 14:33:17 INFO dspy.evaluate.evaluate: Average Metric: 22.796277534287974 / 35 (65.1%)
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.13 on minibatch of size 35 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 7'].
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [60.5, 49.37, 56.45, 56.69, 54.13, 57.35, 59.21, 54.66, 51.92, 54.56, 50.87, 57.91, 55.51, 55.75, 58.66, 57.4, 55.99, 65.13]
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [55.15, 56.26, 57.41, 56.38]
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 57.41
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 23 / 23 - Full Evaluation =====
2025/07/13 14:33:17 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging


Average Metric: 93.15 / 160 (58.2%): 100%|██████████| 160/160 [00:51<00:00,  3.09it/s]

2025/07/13 14:34:09 INFO dspy.evaluate.evaluate: Average Metric: 93.14748450501101 / 160 (58.2%)
2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 58.22
2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [55.15, 56.26, 57.41, 56.38, 58.22]
2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 58.22
2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/13 14:34:09 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 58.22!


In [45]:
baseline = rag(question="cmd+tab does not work on hidden or minimized windows")
print(baseline.response)

By default, cmd+tab on macOS cycles through open applications, but it does not always bring minimized or hidden windows to the foreground. If you try to switch to a minimized window using cmd+tab, the app may become active, but its window will remain minimized. To work around this, you can use third-party apps like Witch (as mentioned in [1]) that offer more control over window management, including options to ignore minimized windows. Additionally, you can adjust Mission Control settings (see [2]) to change how spaces and windows are handled when switching applications. However, the default behavior is that cmd+tab does not restore minimized or hidden windows.


Trace(trace_id=147cf8e6ee394f4582ebcb6708b8bc78)

In [46]:
pred = optimized_rag(question="cmd+tab does not work on hidden or minimized windows")
print(pred.response)

By default, cmd+tab on macOS switches between applications but does not restore or focus minimized or hidden windows. If you switch to an app whose windows are minimized, the app becomes active, but its windows remain minimized. To change this behavior, you can try:
- Going to System Preferences > Mission Control and unchecking "When switching to an application, switch to a Space with open windows for the application."
- Using third-party utilities like Witch for more advanced window management.
Alternatively, you can manually restore minimized windows by clicking their icons in the Dock.


Trace(trace_id=1ba7c468679a4e7eaa8b9383fb64c535)

In [47]:
evaluate(optimized_rag)

Average Metric: 165.61 / 300 (55.2%): 100%|██████████| 300/300 [01:58<00:00,  2.53it/s]

2025/07/13 14:41:09 INFO dspy.evaluate.evaluate: Average Metric: 165.60628529485274 / 300 (55.2%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],The context provides several perspectives on when to use C versus ...,Use C over C++ when: - You are working in embedded systems or envi...,✔️ [0.400]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",The context provides several perspectives on storing images in a g...,"Images can be stored in a git repository, especially if they are a...",✔️ [0.727]


55.2

[Trace(trace_id=427e9e9225f24de184016e2e781fc43a), Trace(trace_id=6d6d519dd4514f1992d0939b8628bf8b), Trace(trace_id=c0a3da2223e74b9788370a81e3165837), Trace(trace_id=90792df09c82438e99983960259a8d0f), Trace(trace_id=016c746fee8b45988f014ed347dc7079), Trace(trace_id=311142affce240f1b5241870eae7dc19), Trace(trace_id=26244c9dcf59492a972f7cd704a713ef), Trace(trace_id=5e03b848747f469b94095c349ea37d6a), Trace(trace_id=7f7ea29b921849289a6ca21af35c8bdd), Trace(trace_id=7f7e7700ff684916bc4e5ece8f73776f)]

In [49]:
cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])  # in USD, as calculated by LiteLLM for certain providers
print(cost)

24.464128


In [50]:
optimized_rag.save("optimized_rag.json")

loaded_rag = RAG()
loaded_rag.load("optimized_rag.json")

loaded_rag(question="cmd+tab does not work on hidden or minimized windows")

Prediction(
    reasoning='The context provides several insights into how macOS handles application switching with cmd+tab, especially regarding minimized or hidden windows. Source [3] explains that when you use cmd+tab to switch to an application, if that application\'s windows are minimized, they do not automatically become visible or focused. You must switch to another app and let it take focus first before you can bring up a minimized window. Additionally, both the Command and Option keys must be pressed on the same side of the keyboard for certain behaviors. Source [2] suggests adjusting a setting in System Preferences > Mission Control by unchecking "When switching to an application, switch to a Space with open windows for the application," which may affect how cmd+tab interacts with minimized windows. Source [1] mentions that cmd+tab cycles between the most recently used applications, but does not bring up the heads-up display if done quickly, and third-party apps like Witch can

Trace(trace_id=2ab89ec5f3bf4402ab2e5addb4cd747c)